In [2]:
#Basics
import os
import cv2
import pandas as pd
import numpy as np

#Sklearn
from sklearn.model_selection import train_test_split

#Keras
from keras.utils import np_utils
from keras.layers import Input
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import concatenate
from keras.layers import Flatten, Dense
from keras.models import Model
from keras.optimizers import SGD
from keras.models import model_from_json
from keras.preprocessing.image import img_to_array

Using TensorFlow backend.


In [35]:
#Carrega dataset com Pandas
df = pd.read_csv('/home/gabriel/Documents/Projetos/SisIntel/Trabalho/list_attr_celeba.csv')

In [56]:
#Para balanceamento o dataset (undersampling)
male = df[df['Attractive'] == 1][['image_id','Attractive']].sample(10000)
female = df[df['Attractive'] == -1][['image_id','Attractive']].sample(10000)

In [57]:
imagePaths = pd.concat([male[['image_id']], female[['image_id']]], axis=0)
labels = pd.concat([male[['Attractive']], female[['Attractive']]])

In [58]:
#Get images
data = []
labels = labels['Attractive'].values.tolist()
for imagePath in imagePaths['image_id']:
    image = cv2.imread("/home/gabriel/Documents/Projetos/SisIntel/Trabalho/img_align_celeba/" + imagePath)
    image = cv2.resize(image, (32, 32))
    image = img_to_array(image)
    data.append(image)

In [59]:
#Divide em train e validation com .8 train e .2 test
X_train, X_test, y_train, y_test = train_test_split(np.array(data), labels, test_size=.2)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

y_train = np.clip(y_train, 0, 1)

y_train = np_utils.to_categorical(y_train, num_classes=2)
y_test = np_utils.to_categorical(y_test)

In [60]:
print (X_train.shape, y_train.shape)

(16000, 32, 32, 3) (16000, 2)


In [61]:
#Modelo
input_img = Input(shape = (32, 32, 3))

tower_1 = Conv2D(64, (3,3), padding='same', activation='relu')(input_img)
tower_1 = Conv2D(64, (3,3), padding='same', strides=(2,2), activation='relu')(tower_1) 

tower_2 = Conv2D(64, (3,3), padding='same', activation='relu')(input_img)
tower_2 = Conv2D(64, (5,5), padding='same', strides=(2,2), activation='relu')(tower_2)

tower_3 = MaxPooling2D((3,3), strides=(1,1), padding='same')(input_img)
tower_3 = Conv2D(64, (3,3), padding='same', strides=(2,2), activation='relu')(tower_3)

output = concatenate([tower_1, tower_2, tower_3], axis = 2)

output = Flatten()(output)
out    = Dense(2, activation='softmax')(output)

model = Model(inputs = input_img, outputs = out)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_86 (Conv2D)              (None, 32, 32, 64)   1792        input_18[0][0]                   
__________________________________________________________________________________________________
conv2d_88 (Conv2D)              (None, 32, 32, 64)   1792        input_18[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_25 (MaxPooling2D) (None, 32, 32, 3)    0           input_18[0][0]                   
__________________________________________________________________________________________________
conv2d_87 

In [62]:
#Configurações
epochs = 5
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['acc'])

In [63]:
#Treino
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32)

Train on 16000 samples, validate on 4000 samples
Epoch 1/5
16000/16000 [==============================] - 170s 11ms/step - loss: 0.5786 - acc: 0.6962 - val_loss: 0.7216 - val_acc: 0.6182
Epoch 2/5
16000/16000 [==============================] - 175s 11ms/step - loss: 0.5176 - acc: 0.7451 - val_loss: 0.9135 - val_acc: 0.5118
Epoch 3/5
16000/16000 [==============================] - 167s 10ms/step - loss: 0.4926 - acc: 0.7586 - val_loss: 1.1641 - val_acc: 0.4363
Epoch 4/5
16000/16000 [==============================] - 178s 11ms/step - loss: 0.4782 - acc: 0.7708 - val_loss: 1.1206 - val_acc: 0.4537
Epoch 5/5
16000/16000 [==============================] - 179s 11ms/step - loss: 0.4694 - acc: 0.7749 - val_loss: 0.9125 - val_acc: 0.5570


In [ ]:
#Salvar pesos
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights(os.path.join(os.getcwd(), 'model.h5'))

In [64]:
#Acurácia
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 55.70%
